In [1]:
from api.BinanceData import BinanceData
from utils.CleanData import CleanData
from variables.Returns import Returns
from variables.Target import Target
from variables.TradingIndicators import TradingIndicators
import pandas as pd
from models.Models import CryptoModel

/Users/noe/Documents/BigData S.Barthelemy/bigdata-project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import de données
BinanceData = BinanceData()
data = BinanceData.load_data("BTCUSDT", days=1095) # 3 ans de données
print(data.head(2))
print(len(data))

       open_time            open            high             low  \
0  1664658000000  19250.95000000  19261.54000000  19248.87000000   
1  1664658300000  19260.30000000  19262.33000000  19244.58000000   

            close        volume     close_time quote_asset_volume  \
0  19259.68000000  269.59853000  1664658299999   5191135.49695430   
1  19250.17000000  238.35668000  1664658599999   4588603.20906890   

   number_of_trades taker_buy_base   taker_buy_quote ignore  
0              5443   169.61903000  3266112.94285090      0  
1              5358   113.42291000  2183540.30969950      0  
315345


In [3]:
# Nettoyage des données 
cleaner = CleanData()
cleaned_data = cleaner.clean_klines_data(data)
print(cleaned_data.index.name)
print(cleaned_data.tail(3))

open_time
                          high        low      close     volume  \
open_time                                                         
2025-09-30 20:50:00  114714.91  114356.12  114695.65  121.35967   
2025-09-30 20:55:00  114723.57  114575.75  114626.36   69.78428   
2025-09-30 21:00:00  114719.97  114626.35  114714.40    9.12324   

                     quote_asset_volume  number_of_trades  taker_buy_quote  
open_time                                                                   
2025-09-30 20:50:00        1.389826e+07             17930     9.499265e+06  
2025-09-30 20:55:00        7.999537e+06             11303     1.561878e+06  
2025-09-30 21:00:00        1.046338e+06              2038     4.682245e+05  


In [6]:
# Export des données pour travailler sur un ficheir Excel plutôt que de requêter l'API Binance à chaque fois
print(len(cleaned_data))
cleaned_data.to_csv("../data/clean_dataset.csv", index=True)

315345


In [2]:
# Import des données depuis le fichier Excel
data = pd.read_csv("../data/clean_dataset.csv", index_col=0)

# Création des variables rendement et la cible
data["return"] = Returns.return_(data['close'])
data["return_10"] = Returns.return_10(data['close'])
data["target"] = Target.compute(data["return"])

# ajout des indicateurs techniques
data = TradingIndicators.add_ema(data, price_col="close", window=12, new_col="EMA_12")
data = TradingIndicators.add_macd(data, price_col="close")
data = TradingIndicators.add_bollinger_bands(data, price_col="close")
data = TradingIndicators.add_rsi(data, price_col="close")
data = TradingIndicators.add_atr(data, high_col="high", low_col="low", close_col="close", window=14, new_col="ATR_14")
data = TradingIndicators.add_high_low_range(data, high_col="high", low_col="low")
data = TradingIndicators.add_buy_pressure(data, high_col="high", low_col="low", close_col="close")
data = TradingIndicators.add_volume_pressure(data, taker_buy_col="taker_buy_quote", total_volume_col="quote_asset_volume")
data = TradingIndicators.add_realized_volatility(data, returns_col="return", window=14)

data.index = pd.to_datetime(data.index)

print(data.head(2))
print(data["target"].value_counts())
print(data.columns)

                         high       low     close     volume  \
open_time                                                      
2022-10-01 21:00:00  19261.54  19248.87  19259.68  269.59853   
2022-10-01 21:05:00  19262.33  19244.58  19250.17  238.35668   

                     quote_asset_volume  number_of_trades  taker_buy_quote  \
open_time                                                                    
2022-10-01 21:00:00        5.191135e+06              5443     3.266113e+06   
2022-10-01 21:05:00        4.588603e+06              5358     2.183540e+06   

                       return  return_10  target  ...      MACD  MACD_Signal  \
open_time                                         ...                          
2022-10-01 21:00:00       NaN        NaN       0  ...  0.000000     0.000000   
2022-10-01 21:05:00 -0.000494        NaN       0  ... -0.758632    -0.151726   

                     Bollinger_Upper  Bollinger_Lower  RSI_14  ATR_14  \
open_time                           

In [ ]:
# Application du modèle xgboost avec validation croisée glissante grâce à la classe CryptoModel
import warnings
warnings.filterwarnings("ignore")   

model = CryptoModel(data, lag=2)
results = model.rolling_xgboost(months_train=3, weeks_test=3, n_trials=50)

[I 2025-09-30 23:51:52,132] A new study created in memory with name: no-name-e738897c-0715-4b80-8926-39fa296980dc


Train: 2022-10-01 to 2023-02-01, Test: 2023-02-01 to 2023-03-01
je suis à la fenêtre n° 1


[I 2025-09-30 23:51:52,509] Trial 0 finished with value: 0.5186025980577627 and parameters: {'max_depth': 4, 'learning_rate': 0.010864077019890029, 'n_estimators': 408, 'subsample': 0.5946511146241841, 'colsample_bytree': 0.5537422776508576, 'gamma': 2, 'min_child_weight': 5}. Best is trial 0 with value: 0.5186025980577627.
[I 2025-09-30 23:51:53,164] Trial 1 finished with value: 0.5001269357705002 and parameters: {'max_depth': 6, 'learning_rate': 0.07389351403908387, 'n_estimators': 482, 'subsample': 0.9211618741256521, 'colsample_bytree': 0.6309696212600993, 'gamma': 0, 'min_child_weight': 3}. Best is trial 0 with value: 0.5186025980577627.
[I 2025-09-30 23:51:53,369] Trial 2 finished with value: 0.5268402042595591 and parameters: {'max_depth': 5, 'learning_rate': 0.05630832150105612, 'n_estimators': 395, 'subsample': 0.9811623492708967, 'colsample_bytree': 0.9594902305381527, 'gamma': 5, 'min_child_weight': 2}. Best is trial 2 with value: 0.5268402042595591.
[I 2025-09-30 23:51:53,5

Train: 2022-10-29 to 2023-02-28, Test: 2023-02-28 to 2023-03-28
je suis à la fenêtre n° 2


[I 2025-09-30 23:52:05,267] Trial 1 finished with value: 0.5124227685026949 and parameters: {'max_depth': 3, 'learning_rate': 0.01760226933579501, 'n_estimators': 211, 'subsample': 0.5967061063243477, 'colsample_bytree': 0.5832213267148934, 'gamma': 1, 'min_child_weight': 2}. Best is trial 0 with value: 0.5401169299664137.
[I 2025-09-30 23:52:05,639] Trial 2 finished with value: 0.5089111486925979 and parameters: {'max_depth': 4, 'learning_rate': 0.018808913448774355, 'n_estimators': 401, 'subsample': 0.8479003861240577, 'colsample_bytree': 0.827918291035657, 'gamma': 2, 'min_child_weight': 6}. Best is trial 0 with value: 0.5401169299664137.
[I 2025-09-30 23:52:05,984] Trial 3 finished with value: 0.4984906155663473 and parameters: {'max_depth': 7, 'learning_rate': 0.1049299259223046, 'n_estimators': 500, 'subsample': 0.9216379664286899, 'colsample_bytree': 0.8414946500051083, 'gamma': 2, 'min_child_weight': 10}. Best is trial 0 with value: 0.5401169299664137.
[I 2025-09-30 23:52:06,29

...

Train: 2025-03-15 to 2025-07-15, Test: 2025-07-15 to 2025-08-12
je suis à la fenêtre n° 33


[I 2025-10-01 00:01:34,829] Trial 0 finished with value: 0.48604651162790696 and parameters: {'max_depth': 7, 'learning_rate': 0.03634262093923464, 'n_estimators': 134, 'subsample': 0.730097614069076, 'colsample_bytree': 0.6247002377432613, 'gamma': 2, 'min_child_weight': 8}. Best is trial 0 with value: 0.48604651162790696.
[I 2025-10-01 00:01:35,093] Trial 1 finished with value: 0.5256591337099812 and parameters: {'max_depth': 8, 'learning_rate': 0.18026393834714538, 'n_estimators': 103, 'subsample': 0.8278945026981117, 'colsample_bytree': 0.6376220227097638, 'gamma': 2, 'min_child_weight': 6}. Best is trial 1 with value: 0.5256591337099812.
[I 2025-10-01 00:01:35,237] Trial 2 finished with value: 0.46237583344672745 and parameters: {'max_depth': 4, 'learning_rate': 0.02464053081486737, 'n_estimators': 77, 'subsample': 0.9342657772972306, 'colsample_bytree': 0.5618032743641703, 'gamma': 1, 'min_child_weight': 2}. Best is trial 1 with value: 0.5256591337099812.
[I 2025-10-01 00:01:35,5

Train: 2025-04-12 to 2025-08-12, Test: 2025-08-12 to 2025-09-09
je suis à la fenêtre n° 34


[I 2025-10-01 00:01:52,829] Trial 0 finished with value: 0.5164284841822402 and parameters: {'max_depth': 5, 'learning_rate': 0.041059179829348605, 'n_estimators': 498, 'subsample': 0.5335417699598857, 'colsample_bytree': 0.8696349027142152, 'gamma': 1, 'min_child_weight': 10}. Best is trial 0 with value: 0.5164284841822402.
[I 2025-10-01 00:01:52,978] Trial 1 finished with value: 0.5076715355345526 and parameters: {'max_depth': 4, 'learning_rate': 0.05073698609634014, 'n_estimators': 165, 'subsample': 0.585786859113457, 'colsample_bytree': 0.6767332405754579, 'gamma': 5, 'min_child_weight': 6}. Best is trial 0 with value: 0.5164284841822402.
[I 2025-10-01 00:01:53,181] Trial 2 finished with value: 0.5068713631298749 and parameters: {'max_depth': 3, 'learning_rate': 0.11216556431832196, 'n_estimators': 239, 'subsample': 0.7479675581961847, 'colsample_bytree': 0.7036157751028902, 'gamma': 2, 'min_child_weight': 10}. Best is trial 0 with value: 0.5164284841822402.
[I 2025-10-01 00:01:53,

           train_start           train_end            test_end  max_depth  \
0  2022-10-01 22:35:00 2023-02-01 22:35:00 2023-03-01 22:35:00          5   
1  2022-10-29 22:35:00 2023-02-28 22:35:00 2023-03-28 22:35:00          3   
2  2022-11-26 22:35:00 2023-03-26 22:35:00 2023-04-23 22:35:00          5   
3  2022-12-24 22:35:00 2023-04-24 22:35:00 2023-05-22 22:35:00          3   
4  2023-01-21 22:35:00 2023-05-21 22:35:00 2023-06-18 22:35:00          3   
5  2023-02-18 22:35:00 2023-06-18 22:35:00 2023-07-16 22:35:00          3   
6  2023-03-18 22:35:00 2023-07-18 22:35:00 2023-08-15 22:35:00          4   
7  2023-04-15 22:35:00 2023-08-15 22:35:00 2023-09-12 22:35:00          5   
8  2023-05-13 22:35:00 2023-09-13 22:35:00 2023-10-11 22:35:00          5   
9  2023-06-10 22:35:00 2023-10-10 22:35:00 2023-11-07 22:35:00          4   
10 2023-07-08 22:35:00 2023-11-08 22:35:00 2023-12-06 22:35:00         10   
11 2023-08-05 22:35:00 2023-12-05 22:35:00 2024-01-02 22:35:00          3   

In [ ]:
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

# Importance des features
importances = model.feature_importances_

feat_importance = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importances
}).sort_values(by="importance", ascending=False)

print(feat_importance)


           train_start           train_end            test_end  max_depth  \
0  2022-10-01 22:35:00 2023-02-01 22:35:00 2023-03-01 22:35:00          5   
1  2022-10-29 22:35:00 2023-02-28 22:35:00 2023-03-28 22:35:00          3   
2  2022-11-26 22:35:00 2023-03-26 22:35:00 2023-04-23 22:35:00          5   
3  2022-12-24 22:35:00 2023-04-24 22:35:00 2023-05-22 22:35:00          3   
4  2023-01-21 22:35:00 2023-05-21 22:35:00 2023-06-18 22:35:00          3   
5  2023-02-18 22:35:00 2023-06-18 22:35:00 2023-07-16 22:35:00          3   
6  2023-03-18 22:35:00 2023-07-18 22:35:00 2023-08-15 22:35:00          4   
7  2023-04-15 22:35:00 2023-08-15 22:35:00 2023-09-12 22:35:00          5   
8  2023-05-13 22:35:00 2023-09-13 22:35:00 2023-10-11 22:35:00          5   
9  2023-06-10 22:35:00 2023-10-10 22:35:00 2023-11-07 22:35:00          4   
10 2023-07-08 22:35:00 2023-11-08 22:35:00 2023-12-06 22:35:00         10   
11 2023-08-05 22:35:00 2023-12-05 22:35:00 2024-01-02 22:35:00          3   

In [7]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

def feature_selection(df, target_col="target", lag=1, top_k=None):
    """
    Évalue l'importance des features sur l'ensemble de la série
    et retourne celles à garder.
    
    df : DataFrame complet avec la cible
    target_col : colonne cible
    lag : décalage cible
    top_k : garder seulement les k meilleures features (optionnel)
    """
    
    # Décalage de la cible
    df[target_col] = df[target_col].shift(-lag)
    df = df.dropna()
    
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    # Scaling
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
    
    # Split train/test (par ex. 80/20)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)
    
    # Fit XGBoost
    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train)
    
    # Importance via gain interne XGBoost
    feat_importance = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    
    print("Importance des variables (XGBoost):")
    print(feat_importance)
    
    # Importance par permutation (plus robuste)
    perm = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
    perm_importance = pd.Series(perm.importances_mean, index=X.columns).sort_values(ascending=False)
    
    print("\nImportance par permutation:")
    print(perm_importance)
    
    # Si on ne veut garder que top_k features
    if top_k:
        selected = feat_importance.head(top_k).index.tolist()
    else:
        selected = feat_importance.index.tolist()
    
    return selected

feature_selection(data, target_col="target", lag=1, top_k=10)

/Users/noe/Documents/BigData S.Barthelemy/bigdata-project/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [23:33:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Importance des variables (XGBoost):
close                  0.054318
low                    0.053604
Bollinger_Lower        0.053576
return_10              0.052452
Bollinger_Upper        0.051329
number_of_trades       0.050565
quote_asset_volume     0.050413
Buy_Pressure           0.050241
MACD                   0.049950
MACD_Signal            0.049650
return                 0.049446
taker_buy_quote        0.049219
volume                 0.049193
EMA_12                 0.049190
Volume_Pressure        0.048792
ATR_14                 0.048734
High_Low_Range         0.048304
RSI_14                 0.048066
Realized_Volatility    0.047382
high                   0.045577
dtype: float32

Importance par permutation:
return_10              0.002821
High_Low_Range         0.002117
RSI_14                 0.001618
ATR_14                 0.001494
high                   0.001194
taker_buy_quote        0.000928
return                 0.000828
close                  0.000528
Volume_Pressure        0

['close',
 'low',
 'Bollinger_Lower',
 'return_10',
 'Bollinger_Upper',
 'number_of_trades',
 'quote_asset_volume',
 'Buy_Pressure',
 'MACD',
 'MACD_Signal']